In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load population dataset

In [2]:
df_pop = pd.read_excel('dataset\WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx')  # Source: https://population.un.org/wpp/Download/Standard/MostUsed/

# Drop the first 15 rows and set the first row as the header
df_pop = df_pop.iloc[15:]
df_pop = df_pop.rename(columns=df_pop.iloc[0]).drop(df_pop.index[0])
df_pop

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,"Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)","Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)","Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)","Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 50 (deaths under age 50 per 1,000 females alive at age 15)","Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)"
16,1,Estimates,WORLD,NaN,900,NaN,NaN,1,World,0,...,580.75,498.04,240.316,271.625,208.192,378.697,430.259,324.931,0,0
17,2,Estimates,WORLD,NaN,900,NaN,NaN,1,World,0,...,566.728,490.199,231.177,258.09,203.78,368.319,415.836,319.336,0,0
18,3,Estimates,WORLD,NaN,900,NaN,NaN,1,World,0,...,546.317,477.264,218.674,240.034,197.142,353.055,395.533,309.91,0,0
19,4,Estimates,WORLD,NaN,900,NaN,NaN,1,World,0,...,535.829,469.532,212.872,232.602,193.049,345.083,385.843,303.905,0,0
20,5,Estimates,WORLD,NaN,900,NaN,NaN,1,World,0,...,523.124,458.484,205.762,224.05,187.444,335.442,374.658,295.994,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20607,20592,Estimates,Wallis and Futuna Islands,2,876,WLF,WF,876,Country/Area,957,...,92.066,102.718,30.643,26.781,33.892,82.852,82.197,83.32,-0.201,-16.706
20608,20593,Estimates,Wallis and Futuna Islands,2,876,WLF,WF,876,Country/Area,957,...,99.196,89.345,29.201,29.159,29.17,79.633,88.556,72.163,-0.171,-14.368
20609,20594,Estimates,Wallis and Futuna Islands,2,876,WLF,WF,876,Country/Area,957,...,77.781,80.136,24.304,22.291,25.981,66.639,69.324,64.496,-0.171,-14.489
20610,20595,Estimates,Wallis and Futuna Islands,2,876,WLF,WF,876,Country/Area,957,...,85.608,86.364,26.572,24.691,28.127,72.73,76.411,69.685,-0.083,-7.096


# Load Powerty and Inequality Platform dataset

In [3]:
# TODO - Consider the population dataset from the World Bank to calculate the poverty rate
# Load the population dataset. Source: https://pip.worldbank.org/
df_pip = pd.read_csv('dataset\pip.csv')  # Poverty and Inequality Platform (PIP) dataset

# Create a dataframe that links country to region
df_region_country = df_pip[['region_name','region_code','country_name','country_code']].drop_duplicates()

In [4]:
# Keep only national data (remove rural or urban only data)
df_pip = df_pip[df_pip['reporting_level'] == 'national']

# Drop duplicates having welfare_type = "consumption"
df_pip = df_pip[~((df_pip[['country_name', 'country_code', 'reporting_year']].duplicated(keep=False)) &
                  (df_pip['welfare_type'] == "consumption"))
                ].reset_index(drop=True)

# Calculate population below 2.15$ a day
df_pip['population below 2.15$ a day (thousands)'] = (
    df_pip['headcount'] *
    df_pip.merge(
        df_pop[['ISO3 Alpha-code', 'Year', 'Total Population, as of 1 January (thousands)']].dropna(),
        how='left',
        left_on=['country_code', 'reporting_year'],
        right_on=['ISO3 Alpha-code', 'Year'],
        validate='one_to_one')
    ['Total Population, as of 1 January (thousands)']
    .astype(float)
    )

# Keep only relevant columns and rename it
df_pip = df_pip[['region_name', 'region_code', 'country_name', 'country_code', 'reporting_year', 'population below 2.15$ a day (thousands)']]
df_pip.rename(columns={'reporting_year':'year'}, inplace=True)

df_pip

,region_name,region_code,country_name,country_code,year,population below 2.15$ a day (thousands)
0,Sub-Saharan Africa,SSA,Angola,AGO,2000,3452.597814
1,Sub-Saharan Africa,SSA,Angola,AGO,2008,3115.663034
2,Sub-Saharan Africa,SSA,Angola,AGO,2018,9566.558328
3,Europe & Central Asia,ECA,Albania,ALB,1996,17.536361
4,Europe & Central Asia,ECA,Albania,ALB,2002,34.297065
...,...,...,...,...,...,...
1948,Sub-Saharan Africa,SSA,Zambia,ZMB,2010,9280.114406
1949,Sub-Saharan Africa,SSA,Zambia,ZMB,2015,9811.004963
1950,Sub-Saharan Africa,SSA,Zimbabwe,ZWE,2011,2789.554026
1951,Sub-Saharan Africa,SSA,Zimbabwe,ZWE,2017,4994.706222


# Create the main dataframe to store all results

In [6]:
# Create the main dataframe to store all results
df = pd.DataFrame(columns=['region_name', 'region_code', 'country_name', 'country_code'])
for year in range(1900, 2023):
    df_ = df_region_country.copy()
    df_['year'] = year
    df = pd.concat([df, df_])
df['year'] = df['year'].astype(int)
df

,region_name,region_code,country_name,country_code,year
0,Sub-Saharan Africa,SSA,Angola,AGO,1900
3,Europe & Central Asia,ECA,Albania,ALB,1900
17,Other High Income Countries,OHI,United Arab Emirates,ARE,1900
19,Latin America & Caribbean,LAC,Argentina,ARG,1900
51,Europe & Central Asia,ECA,Armenia,ARM,1900
...,...,...,...,...,...
2176,Europe & Central Asia,ECA,Kosovo,XKX,2022
2188,Middle East & North Africa,MNA,"Yemen, Rep.",YEM,2022
2191,Sub-Saharan Africa,SSA,South Africa,ZAF,2022
2197,Sub-Saharan Africa,SSA,Zambia,ZMB,2022


In [7]:
# Add the share-of-population-below-2.15$-a-day to the main dataframe
df = df.merge(
    df_pip, 
    how='left', 
    on=['region_name', 'region_code', 'country_name', 'country_code', 'year'],
    validate='one_to_one'
    )

In [9]:
# Add the population to the main dataframe
df['Total Population (thousands)'] = df.merge(
    df_pop.loc[~df_pop['ISO3 Alpha-code'].isna()], 
    how='left', 
    left_on=['country_code', 'year'], 
    right_on=['ISO3 Alpha-code', 'Year'],
    validate='one_to_one'
    )['Total Population, as of 1 January (thousands)'].astype(float)

In [10]:
import seaborn as sns
sns.set_theme(style="darkgrid")

# Grouping by region and year
df_gp = df.groupby(['region_name', 'year']).mean(numeric_only=True)
df_gp.reset_index(inplace=True)

# Adding the world data
df_world = df.groupby('year').mean(numeric_only=True).reset_index()
df_world['region_name'] = 'World'
df_gp = pd.concat([df_gp, df_world])

# Plot the responses for different events and regions
# sns.set(rc={'figure.figsize':(30,20)})
g = sns.lmplot(x="year", y="share-of-population-below-2.15$-a-day",
             hue="region_name", data=pd.concat([df_gp, df_world]))
g.figure.set_size_inches(15, 10)
del g, df_gp, df_world

KeyError: "['share-of-population-below-2.15$-a-day'] not in index"

In [ ]:
df_le = pd.read_csv('dataset\life-expectancy.csv')
df_le

In [ ]:
df.isna().sum()

In [ ]:
df.drop(columns=['Total Population (thousands)'], inplace=True)

In [ ]:
df.groupby('year').sum(numeric_only=True).plot(y='Total Population (thousands)')